# GCN

## 0. Paper

### Info
* Title: Semi-Supervised Classification with Graph Convolutional Networks
* Author: Thomas N. Kipf
* Task: Node Classification
* Link: https://arxiv.org/abs/1609.02907


### Features
* Dataset: cora, [link](https://github.com/tkipf/pygcn/tree/master/data/cora)

### Reference
* https://github.com/tkipf/pygcn


## 1. Setting

In [1]:
import os
import math
from glob import glob
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import scipy.sparse as sp
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchsummary import torchsummary

In [2]:
class CONFIG:
    lr = 0.01
    weight_decay = 5e-4
    drop_rate = 0.5
    hidden_dim = 16
    epoch_size = 200
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dataset = 'cora'
    base_dir = '/content/drive/Shared drives/Yoon/Project/Doing/Deep Learning Paper Implementation'

## 2. Data

In [3]:
def load_data():
    content = np.genfromtxt('data/cora.content', dtype=str)
    feature = sp.csr_matrix(content[:,1:-1], dtype=np.float32)
    index = np.array(content[:,0], dtype=np.int32)
    label = LabelEncoder().fit_transform(content[:,-1])

    index_dict = {j:i for i,j in enumerate(index)}
    edges = np.genfromtxt('data/cora.cites', dtype=int)
    edges = np.array(list(map(index_dict.get, edges.flatten()))).reshape((-1, 2))

    data = np.ones(edges.shape[0]) # the entries of the matrix 
    i, j = edges[:, 0], edges[:, 1] # row/column indices of the matrix entries
    M, N = label.shape[0], label.shape[0]
    adj = sp.coo_matrix((data, (i, j)), shape=(M, N), dtype=np.float32) # A[i[k], j[k]] = data[k]
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj) # build symmetric adjacency matrix (directed -> undirected)

    feature = normalize_row(feature)
    adj = normalize_row(adj + sp.eye(adj.shape[0]))

    train_idx, val_idx, test_idx = range(140), range(200, 500), range(500, 1500)
    feature = torch.FloatTensor(feature.toarray())
    label = torch.LongTensor(label)
    adj = sparse2torch(adj)

    train_idx = torch.LongTensor(train_idx)
    val_idx = torch.LongTensor(val_idx)
    test_idx = torch.LongTensor(test_idx)
    return adj, feature, label, train_idx, val_idx, test_idx

def normalize_row(mx):
    row_sum = np.array(mx.sum(1))
    r_inv = np.power(row_sum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_inv = sp.diags(r_inv)
    mx = r_inv.dot(mx)
    return mx

def sparse2torch(mx):
    mx = mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(np.vstack((mx.row, mx.col)).astype(np.int64))
    values = torch.from_numpy(mx.data)
    shape = torch.Size(mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [ ]:
data_path = os.path.join(CONFIG.base_dir, 'data', CONFIG.dataset)
!unzip -q "{data_path}" -d 'data'

In [4]:
adj, feature, label, train_idx, val_idx, test_idx = load_data()

In [5]:
print('adj: ', adj.size())
print('feature: ', feature.size())
print('label: ', label.size())
print('train_idx: ', train_idx.size())
print('val_idx: ', val_idx.size())
print('test_idx: ', test_idx.size())

adj:  torch.Size([2708, 2708])
feature:  torch.Size([2708, 1433])
label:  torch.Size([2708])
train_idx:  torch.Size([140])
val_idx:  torch.Size([300])
test_idx:  torch.Size([1000])


## 3. Model

In [6]:
class GraphConv(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConv, self).__init__()
        self.in_features= in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.initialize()
    
    def initialize(self):
        std = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-std, std)
        if self.bias is not None:
            self.bias.data.uniform_(-std, std)
    
    def forward(self, x, adj):
        support = torch.mm(x, self.weight) # (num_nodes, out_features)
        outputs = torch.spmm(adj, support) # (num_nodes, out_features)
        if self.bias is not None:
            outputs += self.bias
        return outputs

In [7]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_class, drop_rate):
        super(GCN, self).__init__()
        self.gconv1 = GraphConv(input_dim, hidden_dim)
        self.gconv2 = GraphConv(hidden_dim, n_class)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drop_rate)
    
    def forward(self, x, adj):
        x = self.gconv1(x, adj)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.gconv2(x, adj)
        return x

## 4. Experiment

In [8]:
class AverageMeter(object):
    def __init__(self, name):
        self.name = name
        self.reset()

    def reset(self):
        self.sum = 0
        self.count = 0
        self.avg = 0

    def update(self, val, n=1):
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = f'{self.name:10s} {self.avg:.3f}'
        return fmtstr


class ProgressMeter(object):
    def __init__(self, meters):
        self.meters = [AverageMeter(m) for m in meters]
    
    def reset(self):
        for m in self.meters:
            m.reset()
    
    def update(self, values, n=1):
        for m, v in zip(self.meters, values):
            m.update(v, n)
            self.__setattr__(m.name, m.avg)

    def log(self):
        msg = [str(meter) for meter in self.meters]
        msg = ' | '.join(msg)
        return msg


def accuracy(logits, targets):
    _, preds = logits.max(1)
    acc = (preds == targets).float().mean()
    return acc

In [9]:
class Trainer(object):
    def __init__(self, model, criterion, optimizer):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.best_epoch, self.best_score = 0, 0
        

    def train(self, adj, feature, label, train_idx, epoch):
        progress = ProgressMeter(["train_loss", "train_acc"])
        self.model.train()
        outputs = self.model(feature, adj)
        loss = self.criterion(outputs[train_idx], label[train_idx])
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        loss = loss.item()
        acc = accuracy(outputs[train_idx], label[train_idx]).item()
        progress.update([loss, acc], n=feature.size(0))
        print(f'TRAIN {epoch:03d} | {progress.log()}')
    
    def validate(self, adj, feature, label, val_idx,  epoch):
        progress = ProgressMeter(["valid_loss", "valid_acc"])
        self.model.eval()

        with torch.no_grad():
            outputs = self.model(feature, adj)
            loss = self.criterion(outputs[val_idx], label[val_idx])
            loss = loss.item()
            acc = accuracy(outputs[val_idx], label[val_idx]).item()
        
        progress.update([loss, acc], n=feature.size(0))
        print(f'VALID {epoch:03d} | {progress.log()}')

        if progress.valid_acc > self.best_score:
            self.best_epoch = epoch
            self.best_score = progress.valid_acc
            ckpt = {
                'best_epoch': self.best_epoch,
                'best_score': self.best_score,
                'model_state_dict': self.model.state_dict()
            }
            torch.save(ckpt, 'ckpt.pt')

    
    def test(self, adj, feature, label, test_idx):
        progress = ProgressMeter(["test_loss", "test_acc"])
        ckpt = torch.load('ckpt.pt')
        self.model.load_state_dict(ckpt['model_state_dict'])
        self.model.eval()

        with torch.no_grad():
            outputs = self.model(feature, adj)
            loss = self.criterion(outputs[test_idx], label[test_idx])
            loss = loss.item()
            acc = accuracy(outputs[test_idx], label[test_idx]).item()
        
        progress.update([loss, acc], n=feature.size(0))
        print(f'TEST | {progress.log()}')  

In [10]:
adj, feature, label, train_idx, val_idx, test_idx = load_data()
adj, feature, label, train_idx, val_idx, test_idx = adj.to(CONFIG.device), feature.to(CONFIG.device), label.to(CONFIG.device), train_idx.to(CONFIG.device), val_idx.to(CONFIG.device), test_idx.to(CONFIG.device)
model = GCN(
    input_dim = feature.shape[1],
    hidden_dim = CONFIG.hidden_dim,
    n_class = 7,
    drop_rate = CONFIG.drop_rate
)
model.to(CONFIG.device)

criterion = nn.CrossEntropyLoss().to(CONFIG.device)
optimizer = torch.optim.Adam(model.parameters(), CONFIG.lr, weight_decay=CONFIG.weight_decay)
trainer = Trainer(model, criterion, optimizer)

In [11]:
for ep in range(CONFIG.epoch_size):
    print('=' * 65)
    trainer.train(adj, feature, label, train_idx, ep)
    trainer.validate(adj, feature, label, val_idx, ep)

TRAIN 000 | train_loss 1.983 | train_acc  0.079
VALID 000 | valid_loss 1.930 | valid_acc  0.270
TRAIN 001 | train_loss 1.959 | train_acc  0.179
VALID 001 | valid_loss 1.915 | valid_acc  0.350
TRAIN 002 | train_loss 1.945 | train_acc  0.264
VALID 002 | valid_loss 1.902 | valid_acc  0.350
TRAIN 003 | train_loss 1.929 | train_acc  0.293
VALID 003 | valid_loss 1.890 | valid_acc  0.350
TRAIN 004 | train_loss 1.921 | train_acc  0.293
VALID 004 | valid_loss 1.878 | valid_acc  0.350
TRAIN 005 | train_loss 1.909 | train_acc  0.293
VALID 005 | valid_loss 1.866 | valid_acc  0.350
TRAIN 006 | train_loss 1.894 | train_acc  0.293
VALID 006 | valid_loss 1.856 | valid_acc  0.350
TRAIN 007 | train_loss 1.884 | train_acc  0.293
VALID 007 | valid_loss 1.845 | valid_acc  0.350
TRAIN 008 | train_loss 1.880 | train_acc  0.293
VALID 008 | valid_loss 1.836 | valid_acc  0.350
TRAIN 009 | train_loss 1.868 | train_acc  0.293
VALID 009 | valid_loss 1.827 | valid_acc  0.350
TRAIN 010 | train_loss 1.860 | train_acc

In [12]:
trainer.test(adj, feature, label, test_idx)

TEST | test_loss  0.763 | test_acc   0.814
